# Generative QA with Pretrained QA Encoder

Ноутбук для обучения генеративной QA системы с использованием **предобученного на SQuAD encoder** и компактного decoder.

**Ключевая идея**: 
- Используем DistilBERT, fine-tuned на SQuAD (distilbert-base-cased-distilled-squad)
- Замораживаем encoder и **предвычисляем** все embeddings заранее
- Decoder использует **pretrained embeddings** от DistilBERT
- Обучаем компактный Transformer Decoder (256-dim, 2 слоя, 8 голов)

**Параметры**:
- Encoder max length: 384 токенов (question + context)
- Decoder max length: 32 токена (ответ)
- Предвычисление encoder embeddings (ускорение ~10x)
- Компактный decoder с projection layer
- **Pretrained embeddings** — decoder сразу знает семантику токенов

## Шаг 1: Импорты и настройка

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from datasets import load_dataset
import random
import pytorch_lightning as pl
from transformers import AutoModel, AutoTokenizer
import math
from tqdm.auto import tqdm

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
pl.seed_everything(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

Seed set to 42


PyTorch version: 2.5.1+cu121
Device: cuda


## Шаг 2: Загрузка DistilBERT (fine-tuned на SQuAD) и датасета

In [2]:
# DistilBERT fine-tuned на SQuAD
qa_model_name = "distilbert-base-cased-distilled-squad"

print(f"Loading pretrained QA model: {qa_model_name}")
qa_encoder = AutoModel.from_pretrained(qa_model_name)
tokenizer = AutoTokenizer.from_pretrained(qa_model_name)

# Сохраняем pretrained embeddings для decoder
pretrained_embeddings = qa_encoder.embeddings.word_embeddings.weight.clone().detach()
print(f"Pretrained embeddings shape: {pretrained_embeddings.shape}")

# Замораживаем encoder
for param in qa_encoder.parameters():
    param.requires_grad = False
qa_encoder.eval()
qa_encoder.to(device)

ENCODER_HIDDEN_SIZE = qa_encoder.config.hidden_size  # 768
VOCAB_SIZE = tokenizer.vocab_size

print(f"Encoder loaded and frozen")
print(f"Vocab size: {VOCAB_SIZE}")
print(f"Encoder hidden size: {ENCODER_HIDDEN_SIZE}")

# Загружаем полный SQuAD датасет
print("\nLoading SQuAD dataset...")
dataset = load_dataset("squad")

train_data = dataset['train']
val_data = dataset['validation']

# Полный датасет
train_contexts = train_data['context']
train_questions = train_data['question']
train_answers = train_data['answers']

val_contexts = val_data['context']
val_questions = val_data['question']
val_answers = val_data['answers']

print(f"Train examples: {len(train_contexts)}")
print(f"Val examples: {len(val_contexts)}")

Loading pretrained QA model: distilbert-base-cased-distilled-squad
Pretrained embeddings shape: torch.Size([28996, 768])
Encoder loaded and frozen
Vocab size: 28996
Encoder hidden size: 768

Loading SQuAD dataset...
Train examples: 87599
Val examples: 10570


## Шаг 3: Предвычисление encoder embeddings

In [3]:
MAX_ENCODER_LENGTH = 384  # Увеличили для длинных контекстов
MAX_DECODER_LENGTH = 32

def precompute_encoder_embeddings(
    contexts, questions, answers, encoder, tokenizer,
    max_length=256, max_decoder_length=32, batch_size=64, device='cuda'
):
    encoder.eval()
    
    encoder_embeddings = []
    encoder_masks = []
    decoder_inputs = []
    decoder_targets = []
    decoder_masks = []
    answer_texts = []
    
    num_examples = len(contexts)
    
    with torch.no_grad():
        for start_idx in tqdm(range(0, num_examples, batch_size), desc="Precomputing"):
            end_idx = min(start_idx + batch_size, num_examples)
            
            batch_questions = questions[start_idx:end_idx]
            batch_contexts = contexts[start_idx:end_idx]
            batch_answers = answers[start_idx:end_idx]
            
            encoded = tokenizer(
                list(batch_questions), list(batch_contexts),
                max_length=max_length, padding='max_length',
                truncation=True, return_tensors='pt'
            )
            
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)
            
            outputs = encoder(input_ids=input_ids, attention_mask=attention_mask)
            hidden_states = outputs.last_hidden_state
            
            for i in range(end_idx - start_idx):
                encoder_embeddings.append(hidden_states[i].cpu())
                encoder_masks.append(attention_mask[i].cpu())
                
                answer_text = batch_answers[i]['text'][0]
                answer_texts.append(answer_text)
                
                answer_tokens = tokenizer.encode(answer_text, add_special_tokens=False)
                answer_tokens = answer_tokens[:max_decoder_length - 2]
                
                decoder_input = [tokenizer.cls_token_id] + answer_tokens
                decoder_target = answer_tokens + [tokenizer.sep_token_id]
                
                pad_len = max_decoder_length - len(decoder_input)
                decoder_input = decoder_input + [tokenizer.pad_token_id] * pad_len
                decoder_target = decoder_target + [tokenizer.pad_token_id] * pad_len
                decoder_mask = [1] * (len(answer_tokens) + 1) + [0] * pad_len
                
                decoder_inputs.append(torch.tensor(decoder_input, dtype=torch.long))
                decoder_targets.append(torch.tensor(decoder_target, dtype=torch.long))
                decoder_masks.append(torch.tensor(decoder_mask, dtype=torch.long))
    
    return {
        'encoder_embeddings': encoder_embeddings,
        'encoder_masks': encoder_masks,
        'decoder_inputs': decoder_inputs,
        'decoder_targets': decoder_targets,
        'decoder_masks': decoder_masks,
        'answer_texts': answer_texts
    }

print("Precompute function defined")

Precompute function defined


In [4]:
print("Precomputing train embeddings...")
train_cache = precompute_encoder_embeddings(
    train_contexts, train_questions, train_answers,
    qa_encoder, tokenizer,
    max_length=MAX_ENCODER_LENGTH,
    max_decoder_length=MAX_DECODER_LENGTH,
    batch_size=64, device=device
)

print("\nPrecomputing val embeddings...")
val_cache = precompute_encoder_embeddings(
    val_contexts, val_questions, val_answers,
    qa_encoder, tokenizer,
    max_length=MAX_ENCODER_LENGTH,
    max_decoder_length=MAX_DECODER_LENGTH,
    batch_size=64, device=device
)

print(f"\nTrain cache: {len(train_cache['encoder_embeddings'])} examples")
print(f"Val cache: {len(val_cache['encoder_embeddings'])} examples")

Precomputing train embeddings...


Precomputing:   0%|          | 0/1369 [00:00<?, ?it/s]


Precomputing val embeddings...


Precomputing:   0%|          | 0/166 [00:00<?, ?it/s]


Train cache: 87599 examples
Val cache: 10570 examples


## Шаг 4: Dataset с кэшированными embeddings

In [5]:
class CachedEmbeddingsDataset(Dataset):
    def __init__(self, cache):
        self.encoder_embeddings = cache['encoder_embeddings']
        self.encoder_masks = cache['encoder_masks']
        self.decoder_inputs = cache['decoder_inputs']
        self.decoder_targets = cache['decoder_targets']
        self.decoder_masks = cache['decoder_masks']
        self.answer_texts = cache['answer_texts']
    
    def __len__(self):
        return len(self.encoder_embeddings)
    
    def __getitem__(self, idx):
        return {
            'encoder_hidden_states': self.encoder_embeddings[idx],
            'encoder_attention_mask': self.encoder_masks[idx],
            'decoder_input_ids': self.decoder_inputs[idx],
            'decoder_attention_mask': self.decoder_masks[idx],
            'decoder_target_ids': self.decoder_targets[idx],
            'answer_text': self.answer_texts[idx]
        }

train_dataset = CachedEmbeddingsDataset(train_cache)
val_dataset = CachedEmbeddingsDataset(val_cache)

print(f"Train dataset: {len(train_dataset)}")
print(f"Val dataset: {len(val_dataset)}")

Train dataset: 87599
Val dataset: 10570


## Шаг 5: Компактный Decoder с Pretrained Embeddings

- d_model=256, 2 слоя, 8 голов
- Pretrained embeddings от DistilBERT (768-dim) → projection → 256-dim
- Projection layer для encoder output: 768 → 256

In [6]:
DECODER_DIM = 256
DECODER_HEADS = 8
DECODER_LAYERS = 2  # Уменьшили для компенсации увеличенного encoder
DECODER_FF = 512

class TransformerDecoderLayer(nn.Module):
    def __init__(self, d_model, nhead, dim_feedforward=512, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.cross_attn = nn.MultiheadAttention(d_model, nhead, dropout=dropout, batch_first=True)
        self.linear1 = nn.Linear(d_model, dim_feedforward)
        self.linear2 = nn.Linear(dim_feedforward, d_model)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, tgt, memory, tgt_mask=None, memory_key_padding_mask=None, tgt_key_padding_mask=None):
        tgt2 = self.self_attn(tgt, tgt, tgt, attn_mask=tgt_mask, key_padding_mask=tgt_key_padding_mask)[0]
        tgt = self.norm1(tgt + self.dropout(tgt2))
        tgt2 = self.cross_attn(tgt, memory, memory, key_padding_mask=memory_key_padding_mask)[0]
        tgt = self.norm2(tgt + self.dropout(tgt2))
        tgt2 = self.linear2(self.dropout(F.gelu(self.linear1(tgt))))
        tgt = self.norm3(tgt + self.dropout(tgt2))
        return tgt

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=512):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe.unsqueeze(0))
    
    def forward(self, x):
        return self.dropout(x + self.pe[:, :x.size(1)])

class CompactDecoderQA(nn.Module):
    """Decoder с pretrained embeddings.
    
    - Pretrained embeddings (768-dim) -> projection -> 256-dim
    - Encoder output (768-dim) -> projection -> 256-dim
    """
    def __init__(self, vocab_size, pretrained_embeddings, encoder_dim=768, decoder_dim=256, 
                 nhead=8, num_layers=3, dim_feedforward=512, dropout=0.1):
        super().__init__()
        self.decoder_dim = decoder_dim
        embed_dim = pretrained_embeddings.shape[1]  # 768
        
        # Pretrained embeddings (frozen) + projection
        self.embedding = nn.Embedding.from_pretrained(pretrained_embeddings, freeze=False)
        self.embed_projection = nn.Linear(embed_dim, decoder_dim)
        
        # Encoder output projection
        self.encoder_projection = nn.Linear(encoder_dim, decoder_dim)
        
        # Positional encoding
        self.pos_encoding = PositionalEncoding(decoder_dim, dropout)
        
        # Decoder layers
        self.layers = ...
        
        # Output projection
        self.output_projection = nn.Linear(decoder_dim, vocab_size)
        
    def forward(self, encoder_hidden_states, encoder_attention_mask, decoder_input_ids, decoder_attention_mask=None):
        # Project encoder output: [batch, seq, 768] -> [batch, seq, 256]
        ...
        
        # Embed and project decoder input: [batch, seq] -> [batch, seq, 768] -> [batch, seq, 256]
        x = self.embedding(decoder_input_ids)
        ...
        
        # Causal mask
        seq_len = decoder_input_ids.size(1)
        ...
        
        # Padding masks
        memory_key_padding_mask = (encoder_attention_mask == 0)
        tgt_key_padding_mask = (decoder_attention_mask == 0) if decoder_attention_mask is not None else None
        
        # Decoder layers apply
        ...
        
        return ...

# Подсчет параметров
test_model = CompactDecoderQA(
    vocab_size=VOCAB_SIZE,
    pretrained_embeddings=pretrained_embeddings,
    encoder_dim=ENCODER_HIDDEN_SIZE,
    decoder_dim=DECODER_DIM,
    nhead=DECODER_HEADS,
    num_layers=DECODER_LAYERS,
    dim_feedforward=DECODER_FF
)
num_params = sum(p.numel() for p in test_model.parameters())
trainable_params = sum(p.numel() for p in test_model.parameters() if p.requires_grad)
print(f"Compact Decoder with Pretrained Embeddings")
print(f"Total parameters: {num_params:,} (~{num_params/1e6:.1f}M)")
print(f"Trainable parameters: {trainable_params:,} (~{trainable_params/1e6:.1f}M)")
del test_model

Compact Decoder with Pretrained Embeddings
Total parameters: 31,696,196 (~31.7M)
Trainable parameters: 31,696,196 (~31.7M)


## Шаг 6: Lightning модуль

In [7]:
class CompactDecoderQALightning(pl.LightningModule):
    def __init__(self, vocab_size, pretrained_embeddings, encoder_dim=768, decoder_dim=256, 
                 nhead=8, num_layers=3, dim_feedforward=512, dropout=0.1, lr=3e-4, warmup_steps=500):
        super().__init__()
        self.save_hyperparameters(ignore=['pretrained_embeddings'])
        
        self.model = CompactDecoderQA(
            vocab_size=vocab_size,
            pretrained_embeddings=pretrained_embeddings,
            encoder_dim=encoder_dim,
            decoder_dim=decoder_dim,
            nhead=nhead,
            num_layers=num_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        
        self.lr = lr
        self.warmup_steps = warmup_steps
        self.vocab_size = vocab_size
        self.criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
        
    def forward(self, encoder_hidden_states, encoder_attention_mask, decoder_input_ids, decoder_attention_mask):
        return self.model(encoder_hidden_states, encoder_attention_mask, decoder_input_ids, decoder_attention_mask)
    
    def training_step(self, batch, batch_idx):
        logits = self(batch['encoder_hidden_states'], batch['encoder_attention_mask'],
                      batch['decoder_input_ids'], batch['decoder_attention_mask'])
        loss = self.criterion(logits.view(-1, self.vocab_size), batch['decoder_target_ids'].view(-1))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        logits = self(batch['encoder_hidden_states'], batch['encoder_attention_mask'],
                      batch['decoder_input_ids'], batch['decoder_attention_mask'])
        loss = self.criterion(logits.view(-1, self.vocab_size), batch['decoder_target_ids'].view(-1))
        self.log('val_loss', loss, on_epoch=True, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.lr, weight_decay=0.01)
        total_steps = self.trainer.estimated_stepping_batches
        
        def lr_lambda(step):
            if step < self.warmup_steps:
                return float(step) / float(max(1, self.warmup_steps))
            return max(0.0, float(total_steps - step) / float(max(1, total_steps - self.warmup_steps)))
        
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
        return {'optimizer': optimizer, 'lr_scheduler': {'scheduler': scheduler, 'interval': 'step'}}

print("Lightning module defined")

Lightning module defined


## Шаг 7: Обучение

In [8]:
# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)

print(f"Train batches: {len(train_loader)}")
print(f"Val batches: {len(val_loader)}")

# Model
lightning_model = CompactDecoderQALightning(
    vocab_size=VOCAB_SIZE,
    pretrained_embeddings=pretrained_embeddings,
    encoder_dim=ENCODER_HIDDEN_SIZE,
    decoder_dim=DECODER_DIM,
    nhead=DECODER_HEADS,
    num_layers=DECODER_LAYERS,
    dim_feedforward=DECODER_FF,
    dropout=0.1,
    lr=3e-4,
    warmup_steps=500
)

# Trainer
trainer = pl.Trainer(
    max_epochs=10,
    accelerator='auto',
    devices=1,
    enable_progress_bar=True,
    log_every_n_steps=50,
    check_val_every_n_epoch=1
)

print("\nStarting training...")
trainer.fit(lightning_model, train_loader, val_loader)
print("\nTraining completed!")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA L40') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


Train batches: 685
Val batches: 83

Starting training...


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
Loading `train_dataloader` to estimate number of stepping batches.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable 
  | Name      | Type             | Params | Mode 
-------------------------------------------------------
0 | model     | CompactDecoderQA | 31.7 M | train
1 | criterion | CrossEntropyLoss | 0      | train
-------------------------------------------------------
31.7 M    Trainable params
0         Non-trainable params
31.7 M    Total params
126.785   Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/tam2511/venvs/train_py10/lib/python3.10/site-packages/pytorch_lightning/utilities/data.py:79: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 128. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.



Training completed!


## Шаг 8: Генерация ответов

In [9]:
def generate_answer(decoder_model, encoder_hidden, encoder_mask, tokenizer, max_length=32):
    """Генерирует ответ используя greedy decoding."""
    decoder_model.eval()
    device = encoder_hidden.device
    decoder_ids = torch.tensor([[tokenizer.cls_token_id]], device=device)
    
    with torch.no_grad():
        for _ in range(max_length):
            decoder_mask = torch.ones_like(decoder_ids)
            logits = decoder_model(encoder_hidden, encoder_mask, decoder_ids, decoder_mask)
            next_token = logits[0, -1, :].argmax().unsqueeze(0).unsqueeze(0)
            decoder_ids = torch.cat([decoder_ids, next_token], dim=1)
            if next_token.item() == tokenizer.sep_token_id:
                break
    
    return tokenizer.decode(decoder_ids[0].tolist(), skip_special_tokens=True)


# Тестируем генерацию
lightning_model.eval()
device = next(lightning_model.parameters()).device

print("Testing generation:\n")
print("=" * 60)

for i in range(10):
    sample = val_dataset[i]
    encoder_hidden = sample['encoder_hidden_states'].unsqueeze(0).to(device)
    encoder_mask = sample['encoder_attention_mask'].unsqueeze(0).to(device)
    
    generated = generate_answer(lightning_model.model, encoder_hidden, encoder_mask, tokenizer)
    
    print(f"Ground Truth: {sample['answer_text']}")
    print(f"Generated:    {generated}")
    print("-" * 60)

Testing generation:

Ground Truth: Denver Broncos
Generated:    Vienna Literary Agreement
------------------------------------------------------------
Ground Truth: Carolina Panthers
Generated:    Detroit Newspaper
------------------------------------------------------------
Ground Truth: Santa Clara, California
Generated:    Jan ' s Park
------------------------------------------------------------
Ground Truth: Denver Broncos
Generated:    Boston Globe
------------------------------------------------------------
Ground Truth: gold
Generated:    yellow
------------------------------------------------------------
Ground Truth: "golden anniversary"
Generated:    the golden age
------------------------------------------------------------
Ground Truth: February 7, 2016
Generated:    February 7, 2016
------------------------------------------------------------
Ground Truth: American Football Conference
Generated:    National Football League
--------------------------------------------------

## Шаг 9: Подсчет метрик

Вычисляем стандартные метрики для QA:
- **Exact Match (EM)** — точное совпадение предсказания и ground truth
- **F1 Score** — F1 на уровне токенов (пересечение слов)


In [10]:
import re
import string
from collections import Counter

def normalize_answer(s):
    """Нормализация ответа: lowercase, удаление пунктуации и артиклей."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)
    
    def white_space_fix(text):
        return ' '.join(text.split())
    
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    
    def lower(text):
        return text.lower()
    
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def exact_match_score(prediction, ground_truth):
    """Exact Match: 1 если нормализованные строки совпадают, иначе 0."""
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def f1_score(prediction, ground_truth):
    """F1 Score на уровне токенов."""
    pred_tokens = normalize_answer(prediction).split()
    gt_tokens = normalize_answer(ground_truth).split()
    
    if len(pred_tokens) == 0 or len(gt_tokens) == 0:
        return int(pred_tokens == gt_tokens)
    
    common = Counter(pred_tokens) & Counter(gt_tokens)
    num_same = sum(common.values())
    
    if num_same == 0:
        return 0.0
    
    precision = num_same / len(pred_tokens)
    recall = num_same / len(gt_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    
    return f1

print("Metric functions defined")


Metric functions defined


In [11]:
# Вычисляем метрики на валидационном наборе
lightning_model.eval()
device = next(lightning_model.parameters()).device

em_scores = []
f1_scores = []

# Оцениваем на подмножестве для скорости (генерация медленная)
EVAL_SAMPLES = 1000
eval_indices = random.sample(range(len(val_dataset)), min(EVAL_SAMPLES, len(val_dataset)))

print(f"Evaluating on {len(eval_indices)} samples...")
for i in tqdm(eval_indices):
    sample = val_dataset[i]
    
    encoder_hidden = sample['encoder_hidden_states'].unsqueeze(0).to(device)
    encoder_mask = sample['encoder_attention_mask'].unsqueeze(0).to(device)
    
    generated = generate_answer(lightning_model.model, encoder_hidden, encoder_mask, tokenizer)
    ground_truth = sample['answer_text']
    
    em_scores.append(exact_match_score(generated, ground_truth))
    f1_scores.append(f1_score(generated, ground_truth))

# Результаты
avg_em = np.mean(em_scores) * 100
avg_f1 = np.mean(f1_scores) * 100

print("\n" + "=" * 50)
print("РЕЗУЛЬТАТЫ НА ВАЛИДАЦИОННОМ НАБОРЕ")
print("=" * 50)
print(f"Exact Match (EM): {avg_em:.2f}%")
print(f"F1 Score:         {avg_f1:.2f}%")
print("=" * 50)


Evaluating on 1000 samples...


  0%|          | 0/1000 [00:00<?, ?it/s]


РЕЗУЛЬТАТЫ НА ВАЛИДАЦИОННОМ НАБОРЕ
Exact Match (EM): 17.00%
F1 Score:         27.97%
